In [1]:
from collections import Counter, defaultdict
import string

import pandas as pd

In [2]:
# pd.options.display.max_rows = 999

## Loading data

#### Main repertory

In [3]:
df0 = pd.read_excel('data/src/DunhamsData_Repertory_DatasetSample.xlsx', keep_default_na=False)
df0.columns = [col.strip() for col in df0.columns]
df0.fillna('', inplace=True)

In [4]:
df0.shape

(2, 8)

In [5]:
df0.head()

,Work,Also Known As,Category,Pieces,Part of Larger Work (n/a if stand-alone),Sources (collection/box/folder),Number of events documented as Piece,Number of events documented as Container
0,Americana,America,Container,Veracruzana,n/a,SIU Programs Box 85-86; Rhoda Winter Russell P...,0,52
1,Veracruzana,,Piece/Standalone,,n/a; Americana,SIU Programs Box 85-86; Rhoda Winter Russell P...,65,0


In [6]:
for a, b in zip(string.ascii_uppercase, df0.columns):
    print(a, b)

A Work
B Also Known As
C Category
D Pieces
E Part of Larger Work (n/a if stand-alone)
F Sources (collection/box/folder)
G Number of events documented as Piece
H Number of events documented as Container


#### Dances in dances

In [7]:
df1 = pd.read_excel('data/src/DunhamsData_Repertory_DatasetSample.xlsx', sheet_name=1)
df1.columns = [col.strip() for col in df1.columns]
df1.fillna('', inplace=True)

In [8]:
df1.shape

(1, 4)

In [9]:
df1.head()

,Dances in Dances,Appears in,Sources (collection/box/folder),Number of events
0,La Bamba,Veracruzana,SIU Programs Box 85-86,15


In [10]:
for a, b in zip(string.ascii_uppercase, df1.columns):
    print(a, b)

A Dances in Dances
B Appears in
C Sources (collection/box/folder)
D Number of events


#### Shows

In [11]:
df2 = pd.read_excel('data/src/DunhamsData_Repertory_DatasetSample.xlsx', sheet_name=2)
df2.columns = [col.strip() for col in df2.columns]
df2.fillna('', inplace=True)

In [12]:
df2.shape

(2, 5)

In [13]:
df2.head()

,Name,Also Known As,Pieces,Sources (collection/box/folder),Number of events
0,Caribbean Rhapsody,Rapsodia Caribe; A Caribbean Rhapsody; Rhapsod...,Americana; Veracruzana,SIU Programs Box 85-86,20
1,Tropical Carnival,,Veracruzana; Americana,SIU Programs Box 85-86,4


In [14]:
for a, b in zip(string.ascii_uppercase, df2.columns):
    print(a, b)

A Name
B Also Known As
C Pieces
D Sources (collection/box/folder)
E Number of events


## Pre-processing data

#### Main repertory

In [15]:
col_0_work = 'Work'
col_0_aka = 'Also Known As'
col_0_pieces = 'Pieces'
col_0_larger = 'Part of Larger Work (n/a if stand-alone)'

#### Dances in dances

In [16]:
col_1_dance = 'Dances in Dances'
col_1_appears = 'Appears in'

#### Shows

In [17]:
col_2_show = 'Name'
col_2_aka = 'Also Known As'
col_2_pieces = 'Pieces'

#### New data

In [18]:
# main_df = df0[[col_0_work, col_0_aka, col_0_pieces, col_0_larger]].copy()
# dances_df = df1[[col_1_dance, col_1_appears]].copy()
# shows_df = df2[[col_2_show, col_2_aka, col_2_pieces]].copy()

main_df = df0.copy()
dances_df = df1.copy()
shows_df = df2.copy()

In [19]:
def str2set(x):
    return set(y.strip() for y in x.split(';') if y.strip())

def set2str(x):
    return '; '.join(sorted(x))

In [20]:
main_df[col_0_work] = main_df[col_0_work].apply(str.strip)
main_df[col_0_aka] = main_df[col_0_aka].apply(str2set)
main_df[col_0_pieces] = main_df[col_0_pieces].apply(str2set)
main_df[col_0_larger] = main_df[col_0_larger].apply(str2set)

In [21]:
dances_df[col_1_dance] = dances_df[col_1_dance].apply(str.strip)
dances_df[col_1_appears] = dances_df[col_1_appears].apply(str2set)

In [22]:
shows_df[col_2_show] = shows_df[col_2_show].apply(str.strip)
shows_df[col_2_aka] = shows_df[col_2_aka].apply(str2set)
shows_df[col_2_pieces] = shows_df[col_2_pieces].apply(str2set)

## Checking data integrity

#### Unique ids

In [23]:
all_ids = []
for i, row in main_df.iterrows():
    work = row[col_0_work]
    aka = row[col_0_aka]
    all_ids.append(work)
    for a in aka:
        all_ids.append(a)
for dance in dances_df[col_1_dance]:
    all_ids.append(dance)
for i, row in shows_df.iterrows():
    show = row[col_2_show]
    aka = row[col_2_aka]
    all_ids.append(show)
    for a in aka:
        all_ids.append(a)

In [24]:
len(all_ids) == len(set(all_ids))

True

In [25]:
c = Counter(all_ids)
[(a, b) for (a, b) in c.most_common() if b > 1]

[]

#### Preliminary cleaning

In [26]:
# dances_df[col_1_dance] = dances_df[col_1_dance].replace('Blues', 'Blues [Dances in Dances]')
# dances_df[col_1_dance] = dances_df[col_1_dance].replace('Mambo', 'Mambo [Dances in Dances]')
# dances_df[col_1_dance] = dances_df[col_1_dance].replace('Maracatu', 'Maracatu [Dances in Dances]')
# shows_df = shows_df[shows_df[col_2_show] != 'Island Song']

#### Removing unnecessary duplicated AKAs

In [27]:
# It will affect "Strutters' Ball" only
for i, row in main_df.iterrows():
    work = row[col_0_work]
    aka = row[col_0_aka]
    if work in aka:
        main_df.loc[i, col_0_aka].remove(work)
# for i, row in shows_df.iterrows():
#     show = row[col_2_show]
#     aka = row[col_2_aka]
#     if show in aka:
#         shows_df.loc[i, col_2_aka].remove(show)

#### Renaming ambiguous AKAs

In [28]:
# It will affect 'Bel Congo', 'Congo Femme', 'Congo Fran', 'Mambo', and 'Spanish Harlem'
for i, row in main_df.iterrows():
    work = row[col_0_work]
    aka = row[col_0_aka]
    for a in aka:
        if c[a] > 1:
            new_a = '{} [[AKA for {}]]'.format(a, work)
            print(new_a)
            main_df.loc[i, col_0_aka].remove(a)
            main_df.loc[i, col_0_aka].add(new_a)
# for i, row in shows_df.iterrows():
#     show = row[col_2_show]
#     aka = row[col_2_aka]
#     for a in aka:
#         if c[a] > 1:
#             new_a = '{} [[{}]]'.format(a, show)
#             shows_df.loc[i, col_2_aka].remove(a)
#             shows_df.loc[i, col_2_aka].add(new_a)

#### Unique ids

In [29]:
all_ids = []
for i, row in main_df.iterrows():
    work = row[col_0_work]
    aka = row[col_0_aka]
    all_ids.append(work)
    for a in aka:
        all_ids.append(a)
for dance in dances_df[col_1_dance]:
    all_ids.append(dance)
for i, row in shows_df.iterrows():
    show = row[col_2_show]
    aka = row[col_2_aka]
    all_ids.append(show)
    for a in aka:
        all_ids.append(a)

In [30]:
len(all_ids) == len(set(all_ids))

True

In [31]:
c = Counter(all_ids)
[(a, b) for (a, b) in c.most_common() if b > 1]

[]

#### Smaller and larger work references

In [32]:
main_ids = main_df.Work.unique()

for i, row in main_df.iterrows():
    pieces = list(row[col_0_pieces])
    larger = list(row[col_0_larger])
    for p in pieces:
        if False:
            pass
#         if p == 'Strutters Ball':
#             new_p = "Strutters' Ball"
#             main_df.loc[i, col_0_pieces].remove(p)
#             main_df.loc[i, col_0_pieces].add(new_p)
#         elif p == 'Variations on the Theme "Boogie Woogie"':
#             new_p = 'Variation on the Theme "Boogie Woogie"'
#             main_df.loc[i, col_0_pieces].remove(p)
#             main_df.loc[i, col_0_pieces].add(new_p)
        elif p not in main_ids:
            print('Main - Piece:', p)
    for lw in larger:
        if lw == 'n/a':
            pass
        elif lw not in main_ids:
            print('Main - Larger:', lw)

for i, row in dances_df.iterrows():
    pieces = list(row[col_1_appears])
    for p in pieces:
        if False:
            pass
#         if p == "Bre'r Rabbit An' De Tah Baby":
#             new_p = "Bre'r Rabbit an' de Tah Baby"
#             dances_df.loc[i, col_1_appears].remove(p)
#             dances_df.loc[i, col_1_appears].add(new_p)
#         elif p == 'Mambo - Havana':
#             new_p = 'Mambo'
#             dances_df.loc[i, col_1_appears].remove(p)
#             dances_df.loc[i, col_1_appears].add(new_p)
#         elif p == 'Washerwomen':
#             new_p = 'Washerwomen Dance'
#             dances_df.loc[i, col_1_appears].remove(p)
#             dances_df.loc[i, col_1_appears].add(new_p)
#         elif p == 'Island Songs':
#             new_p = 'Island Song'
#             dances_df.loc[i, col_1_appears].remove(p)
#             dances_df.loc[i, col_1_appears].add(new_p)
        elif p not in main_ids:
            print('Dances - Piece:', p)

for i, row in shows_df.iterrows():
    pieces = list(row[col_2_pieces])
    for p in pieces:
        if False:
            pass
#         if p == 'Shango Ritual and Dance':
#             new_p = 'Shango'
#             shows_df.loc[i, col_2_pieces].remove(p)
#             shows_df.loc[i, col_2_pieces].add(new_p)
#         elif p == 'Strutters Ball':
#             new_p = "Strutters' Ball"
#             shows_df.loc[i, col_2_pieces].remove(p)
#             shows_df.loc[i, col_2_pieces].add(new_p)
#         elif p == 'Marrakech':
#             new_p = 'Marrakech!'
#             shows_df.loc[i, col_2_pieces].remove(p)
#             shows_df.loc[i, col_2_pieces].add(new_p)
#         elif p == 'The Diamond Theif':
#             new_p = 'The Diamond Thief'
#             shows_df.loc[i, col_2_pieces].remove(p)
#             shows_df.loc[i, col_2_pieces].add(new_p)
#         elif p == 'Shango Dance and Ritual':
#             new_p = 'Shango'
#             shows_df.loc[i, col_2_pieces].remove(p)
#             shows_df.loc[i, col_2_pieces].add(new_p)
#         elif p == 'Shango Ritual':
#             new_p = 'Shango'
#             shows_df.loc[i, col_2_pieces].remove(p)
#             shows_df.loc[i, col_2_pieces].add(new_p)
#         elif p == 'Rites De Passage':
#             new_p = 'Rites de Passage'
#             shows_df.loc[i, col_2_pieces].remove(p)
#             shows_df.loc[i, col_2_pieces].add(new_p)
#         elif p == 'La Comparsa':
#             new_p = 'Comparsa'
#             shows_df.loc[i, col_2_pieces].remove(p)
#             shows_df.loc[i, col_2_pieces].add(new_p)
#         elif p == "L'ag'ya":
#             new_p = "L'Ag'Ya"
#             shows_df.loc[i, col_2_pieces].remove(p)
#             shows_df.loc[i, col_2_pieces].add(new_p)
#         elif p == 'Street Scene Port Au Prince':
#             new_p = 'Street Scene (Port Au Prince)'
#             shows_df.loc[i, col_2_pieces].remove(p)
#             shows_df.loc[i, col_2_pieces].add(new_p)
#         elif p == 'Rumba with a Little Jive Mixed in':
#             new_p = 'Rhumba with a Little Jive Mixed In'
#             shows_df.loc[i, col_2_pieces].remove(p)
#             shows_df.loc[i, col_2_pieces].add(new_p)
#         elif p == 'Plantation Dance':
#             new_p = 'Plantation Dances'
#             shows_df.loc[i, col_2_pieces].remove(p)
#             shows_df.loc[i, col_2_pieces].add(new_p)
#         elif p == 'Biguine- Biguine':
#             new_p = 'Biguine, Biguine'
#             shows_df.loc[i, col_2_pieces].remove(p)
#             shows_df.loc[i, col_2_pieces].add(new_p)
#         elif p == 'Sarabande to Mademe Christophe Queen of Haiti':
#             new_p = 'Sarabande to Madame Christophe Queen of Haiti'
#             shows_df.loc[i, col_2_pieces].remove(p)
#             shows_df.loc[i, col_2_pieces].add(new_p)
#         elif p == 'Rhumba Santiago Da Cuba':
#             new_p = 'Rhumba, Santiago da Cuba'
#             shows_df.loc[i, col_2_pieces].remove(p)
#             shows_df.loc[i, col_2_pieces].add(new_p)
#         elif p == 'Bahianna':
#             new_p = 'Bahiana'
#             shows_df.loc[i, col_2_pieces].remove(p)
#             shows_df.loc[i, col_2_pieces].add(new_p)
#         elif p == 'Plantation Dances from':
#             new_p = 'Plantation Dances'
#             shows_df.loc[i, col_2_pieces].remove(p)
#             shows_df.loc[i, col_2_pieces].add(new_p)
#         elif p == 'Tropics - Shore Excursion':
#             new_p = 'Tropics'
#             shows_df.loc[i, col_2_pieces].remove(p)
#             shows_df.loc[i, col_2_pieces].add(new_p)
#         elif p == 'Island Songs':
#             new_p = 'Island Song'
#             shows_df.loc[i, col_2_pieces].remove(p)
#             shows_df.loc[i, col_2_pieces].add(new_p)
        elif p not in main_ids:
            print('Shows - Piece:', p)

#### Others

In [33]:
main_df['Category'] = 'Main'
shows_df['Category'] = 'Show'
dances_df['Category'] = 'Dance'

In [34]:
# main_df.columns = ['Work', 'Aka', 'Pieces', 'Larger_works', 'Category']
# dances_df.columns = ['Dance', 'Pieces', 'Category']
# shows_df.columns = ['Show', 'Aka', 'Pieces', 'Category']

main_df.rename(columns={
    'Work':'Work',
    'Also Known As':'Aka',
    'Pieces':'Pieces',
    'Part of Larger Work (n/a if stand-alone)':'Larger_works',
    'Category':'Category',
}, inplace=True)

dances_df.rename(columns={
    'Dances in Dances (check HB/KE but use TMU as base)':'Dance',
    'Appears in':'Pieces',
    'Category':'Category',
}, inplace=True)

shows_df.rename(columns={
    'Name':'Show',
    'Also Known As':'Aka',
    'Pieces':'Pieces',
    'Category':'Category',
}, inplace=True)

## Saving data

In [35]:
main_df.to_csv('data/out/rep_main_structure.csv', index=False)
shows_df.to_csv('data/out/rep_shows_structure.csv', index=False)
dances_df.to_csv('data/out/rep_dances_structure.csv', index=False)